In [1]:
%run utils.ipynb

import pandas as pd
import numpy as np
from tqdm import tqdm
from annoy import AnnoyIndex
import os
import warnings
import multitasking
import signal

multitasking.set_max_threads(10)
multitasking.set_engine('process')
signal.signal(signal.SIGINT, multitasking.killall)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('precision', 10)

warnings.filterwarnings('ignore')

In [2]:
df_qtime = pd.read_pickle('../user_data/data/qtime.pkl')
df_click = pd.read_pickle('../user_data/data/click.pkl')

In [3]:
phases = sorted(list(df_qtime['phase'].unique()))
phases

[0]

In [4]:
df_item = pd.read_csv('../data/underexpose_train/underexpose_item_feat.csv',
                      header=None)
df_item.columns = ['item_id'] + ['txt_vec' + str(i) for i in range(128)
                                 ] + ['img_vec' + str(i) for i in range(128)]
df_item['txt_vec0'] = df_item['txt_vec0'].apply(lambda x: float(x[1:]))
df_item['txt_vec127'] = df_item['txt_vec127'].apply(lambda x: float(x[:-1]))
df_item['img_vec0'] = df_item['img_vec0'].apply(lambda x: float(x[1:]))
df_item['img_vec127'] = df_item['img_vec127'].apply(lambda x: float(x[:-1]))
df_item.drop_duplicates(['item_id'], inplace=True)
df_item.head()

,item_id,txt_vec0,txt_vec1,txt_vec2,txt_vec3,txt_vec4,txt_vec5,txt_vec6,txt_vec7,txt_vec8,txt_vec9,txt_vec10,txt_vec11,txt_vec12,txt_vec13,txt_vec14,txt_vec15,txt_vec16,txt_vec17,txt_vec18,txt_vec19,txt_vec20,txt_vec21,txt_vec22,txt_vec23,txt_vec24,txt_vec25,txt_vec26,txt_vec27,txt_vec28,txt_vec29,txt_vec30,txt_vec31,txt_vec32,txt_vec33,txt_vec34,txt_vec35,txt_vec36,txt_vec37,txt_vec38,txt_vec39,txt_vec40,txt_vec41,txt_vec42,txt_vec43,txt_vec44,txt_vec45,txt_vec46,txt_vec47,txt_vec48,txt_vec49,txt_vec50,txt_vec51,txt_vec52,txt_vec53,txt_vec54,txt_vec55,txt_vec56,txt_vec57,txt_vec58,txt_vec59,txt_vec60,txt_vec61,txt_vec62,txt_vec63,txt_vec64,txt_vec65,txt_vec66,txt_vec67,txt_vec68,txt_vec69,txt_vec70,txt_vec71,txt_vec72,txt_vec73,txt_vec74,txt_vec75,txt_vec76,txt_vec77,txt_vec78,txt_vec79,txt_vec80,txt_vec81,txt_vec82,txt_vec83,txt_vec84,txt_vec85,txt_vec86,txt_vec87,txt_vec88,txt_vec89,txt_vec90,txt_vec91,txt_vec92,txt_vec93,txt_vec94,txt_vec95,txt_vec96,txt_vec97,txt_vec98,txt_vec99,txt_vec100,txt_vec101,txt_vec102,txt_vec103,txt_vec104,txt_vec105,txt_vec106,txt_vec107,txt_vec108,txt_vec109,txt_vec110,txt_vec111,txt_vec112,txt_vec113,txt_vec114,txt_vec115,txt_vec116,txt_vec117,txt_vec118,txt_vec119,txt_vec120,txt_vec121,txt_vec122,txt_vec123,txt_vec124,txt_vec125,txt_vec126,txt_vec127,img_vec0,img_vec1,img_vec2,img_vec3,img_vec4,img_vec5,img_vec6,img_vec7,img_vec8,img_vec9,img_vec10,img_vec11,img_vec12,img_vec13,img_vec14,img_vec15,img_vec16,img_vec17,img_vec18,img_vec19,img_vec20,img_vec21,img_vec22,img_vec23,img_vec24,img_vec25,img_vec26,img_vec27,img_vec28,img_vec29,img_vec30,img_vec31,img_vec32,img_vec33,img_vec34,img_vec35,img_vec36,img_vec37,img_vec38,img_vec39,img_vec40,img_vec41,img_vec42,img_vec43,img_vec44,img_vec45,img_vec46,img_vec47,img_vec48,img_vec49,img_vec50,img_vec51,img_vec52,img_vec53,img_vec54,img_vec55,img_vec56,img_vec57,img_vec58,img_vec59,img_vec60,img_vec61,img_vec62,img_vec63,img_vec64,img_vec65,img_vec66,img_vec67,img_vec68,img_vec69,img_vec70,img_vec71,img_vec72,img_vec73,img_vec74,img_vec75,img_vec76,img_vec77,img_vec78,img_vec79,img_vec80,img_vec81,img_vec82,img_vec83,img_vec84,img_vec85,img_vec86,img_vec87,img_vec88,img_vec89,img_vec90,img_vec91,img_vec92,img_vec93,img_vec94,img_vec95,img_vec96,img_vec97,img_vec98,img_vec99,img_vec100,img_vec101,img_vec102,img_vec103,img_vec104,img_vec105,img_vec106,img_vec107,img_vec108,img_vec109,img_vec110,img_vec111,img_vec112,img_vec113,img_vec114,img_vec115,img_vec116,img_vec117,img_vec118,img_vec119,img_vec120,img_vec121,img_vec122,img_vec123,img_vec124,img_vec125,img_vec126,img_vec127
0,42844,4.5149450302,-2.3837196827,0.5004140139,0.4070682824,-1.9952288866,0.1090780124,-0.6917753220,2.2274599075,-6.4379744530,-0.8248971701,-0.1387242377,-0.3793291748,0.6276600957,0.4183767140,4.4412183762,0.2998193502,0.5785568357,-4.6992893219,-0.3947401643,-2.3916513920,0.3705316782,-1.3554661274,-1.0741775036,-2.3216402531,-0.3324561715,0.1238859445,-2.4391555786,-0.3455986977,-3.3043470383,1.4852843285,0.9098017812,-1.6430019140,5.0370340347,2.7801148891,4.7764964104,2.2552747726,3.7697074413,-3.6616835594,-0.6494054794,4.1996359825,-0.6348056197,2.4303400517,-2.8740186691,-0.7861775160,-0.5049162507,-6.0077886581,1.4984948635,1.5306128263,2.3796546459,-0.0231464375,-0.7036052942,2.1469361782,3.9448320866,-3.0985984802,2.5134534836,-5.3958601952,0.7621323466,2.8018035889,0.8779643178,-0.5859798193,1.9625556469,-5.1712174416,1.1816873550,0.5419528484,0.4858333170,4.2125682831,3.4581983089,1.4029390812,-0.5462926030,2.1821005344,-0.0962546766,1.7366081476,0.6851462722,-1.6132848263,-0.9128761888,-0.5323970318,1.7248421907,2.2641146183,-2.2597558498,4.5065088272,-0.3460883200,0.9281976819,-2.0782611370,-0.1955756247,4.8475780487,-2.9586863518,1.0060796738,2.0822322369,-2.2596824169,-2.0333144665,0.2182324231,3.9753122330,2.0034470558,-0.2406283617,-2.4685480595,1.6660641432,-2.0819606781,-2.4968724251,-1.9016369581,0.2714739740,-2.2981684208,-1.7873090506,1.618066906

In [5]:
df_img_vec = df_item[['item_id'] + ['img_vec' + str(i) for i in range(128)]]
df_img_vec['item_img_vec'] = df_img_vec.apply(
    lambda x: np.array(x.values[1:].tolist()), axis=1)
items = df_img_vec['item_id'].values.tolist()
img_vecs = df_img_vec['item_img_vec'].values.tolist()
df_img_vec[['item_id',
            'item_img_vec']].to_pickle('../user_data/model/item_img_vec.pkl')

item_vec_map = dict(zip(items, img_vecs))

In [6]:
# 将 item embedding 建立索引
name = 'img'
index_to_item_dict = {}
item_to_index_dict = {}

item_index = AnnoyIndex(128, 'angular')
item_index.set_seed(2020)

for i, row in tqdm(df_item.iterrows()):
    emb = row[-128:].values

    item = row[0]
    index_to_item_dict[i] = item
    item_to_index_dict[item] = i

    item_index.add_item(i, emb)

item_index.build(100)

108916it [00:27, 3937.99it/s]


True

In [7]:
def recall(df_qtime, user_embs, item_index, index_to_item_dict):
    data_list = []
    for user_id, query_time, item_id, phase in tqdm(df_qtime.values):
        user_emb = user_embs[user_id]
        ids, distances = item_index.get_nns_by_vector(user_emb,
                                                      100,
                                                      include_distances=True)

        item_ids = [index_to_item_dict[id] for id in ids]
        # 返回的是距离，距离越小越相似
        item_sim_scores = [2 - distance for distance in distances]

        df_temp = pd.DataFrame()
        df_temp['item_id'] = item_ids
        df_temp['sim_score'] = item_sim_scores
        df_temp['user_id'] = user_id
        df_temp['query_time'] = query_time
        df_temp['phase'] = phase

        if item_id == -1:
            df_temp['label'] = np.nan
        else:
            df_temp['label'] = 0
            df_temp.loc[df_temp['item_id'] == item_id, 'label'] = 1

        df_temp.sort_values(['sim_score'], inplace=True, ascending=False)

        df_temp = df_temp[[
            'user_id', 'phase', 'query_time', 'item_id', 'sim_score', 'label'
        ]]
        df_temp['user_id'] = df_temp['user_id'].astype('int')
        df_temp['item_id'] = df_temp['item_id'].astype('int')

        data_list.append(df_temp)

    df_data = pd.concat(data_list, sort=False)
    return df_data

In [8]:
@multitasking.task
def work(phase, force=False):
    os.makedirs('../user_data/model/recall_v4', exist_ok=True)

    if force or (not os.path.exists(
            '../user_data/model/recall_v4/user_img_vec{}.pkl'.format(phase))
                 or not os.path.exists(
                     '../user_data/model/recall_v4/recall_{}.pkl'.format(phase))):
        # 获取当前阶段的click
        df_click_phase = df_click[df_click['phase'] == phase]

        # 生成用户embedding
        tmp = df_click_phase.groupby('user_id', as_index=False)['item_id'].agg(
            {'list': list})
        sentences = tmp['list'].values.tolist()
        del tmp['list']

        emb_matrix = []
        for seq in sentences:
            seq = seq[::-1]

            vec = []
            for pos, w in enumerate(seq):
                if w in item_vec_map.keys():
                    vec.append(np.asarray(item_vec_map[w]) * (0.7**pos))
            if len(vec) > 0:
                emb_matrix.append(np.mean(vec, axis=0))
            else:
                emb_matrix.append([0] * 128)

        df_user_img_vec = tmp
        df_user_img_vec['user_img_vec'] = emb_matrix
        df_user_img_vec.to_pickle('../user_data/model/user_img_vec.pkl')
        df_user_img_vec['phase'] = phase
        df_user_img_vec.to_pickle(
            '../user_data/model/recall_v4/user_img_vec{}.pkl'.format(phase))

        users = tmp['user_id'].values.tolist()
        user_embs = dict(zip(users, emb_matrix))

        # 获取当前阶段的qtime, 召回
        df_qtime_phase = df_qtime[df_qtime['phase'] == phase]
        df_data = recall(df_qtime_phase, user_embs, item_index,
                         index_to_item_dict)
        df_data.to_pickle('../user_data/model/recall_v4/recall_{}.pkl'.format(phase))

In [9]:
df_user_img_vec_phase = pd.DataFrame()
df_recall = pd.DataFrame()
val_score = np.array([0.0, 0.0, 0.0, 0.0])
force = False

for phase in phases:
    work(phase, force)

multitasking.wait_for_tasks()
print('合并任务')

for phase in phases:
    df_user_img_vec = pd.read_pickle(
        '../user_data/model/recall_v4/user_img_vec{}.pkl'.format(phase))
    df_data = pd.read_pickle('../user_data/model/recall_v4/recall_{}.pkl'.format(phase))

    df_user_img_vec_phase = df_user_img_vec_phase.append(df_user_img_vec)
    df_recall = df_recall.append(df_data)

    score = evaluate_scores(df_data, phase)
    val_score += score

    print('phase', phase, score)

合并任务


100%|██████████| 18505/18505 [00:18<00:00, 988.14it/s]

phase 0 (0.006373214961965345, 0.02024700154375965, 0.00775978719651603, 0.02407566638005159)


In [10]:
df_user_img_vec_phase.to_pickle('../user_data/model/user_img_vec.pkl')

In [11]:
val_score

array([0.00637321, 0.020247  , 0.00775979, 0.02407567])

In [12]:
df_recall.sort_values(['user_id', 'phase', 'query_time'], inplace=True)
df_recall.to_pickle('../user_data/data/recall_v4.pkl')
df_recall.head()

,user_id,phase,query_time,item_id,sim_score,label
0,1,0.0,0.9839419315,17887,1.5567105114,0.0
1,1,0.0,0.9839419315,17559,1.5547945201,0.0
2,1,0.0,0.9839419315,37316,1.5266568065,0.0
3,1,0.0,0.9839419315,31443,1.5244505703,0.0
4,1,0.0,0.9839419315,35217,1.5183159113,0.0
